## Connect To Elastic Search

In [1]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

client_info = es.info()
print("Connected to Elasticsearch!")

pprint(client_info.body)

Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'T1HeaWnRTOqX_BBgREVVbA',
 'name': '4cfd098368b9',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-10-21T10:06:21.288851013Z',
             'build_flavor': 'default',
             'build_hash': '25d88452371273dd27356c98598287b669a03eae',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '10.3.1',
             'minimum_index_compatibility_version': '8.0.0',
             'minimum_wire_compatibility_version': '8.19.0',
             'number': '9.2.0'}}


Crete Index with auto mappings

In [2]:
es.indices.delete(index="my_index", ignore_unavailable=True)
es.indices.create(index="my_index")

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

We are using the dummy_data.json file to insert multiple documents in the index. We store the id of each document in the document_ids list.

In [3]:
from tqdm import tqdm # tqdm = taqaddum (arabic word means progress)
import json 

document_ids = []
documents = json.load(open("data.json"))

for document in tqdm(documents, total=len(documents)):
    response = es.index(index="my_index", body=document)
    document_ids.append(response["_id"])

100%|██████████| 5/5 [00:00<00:00,  9.40it/s]


In [4]:
document_ids

['sKYOPJoBbrc3W2pBVbKQ',
 'saYOPJoBbrc3W2pBVrLN',
 'sqYOPJoBbrc3W2pBVrL2',
 's6YOPJoBbrc3W2pBV7I2',
 'tKYOPJoBbrc3W2pBV7Js']

**Get operation**
#
This is an example of a successful operation. If the ID exists in the document, the get operation won't return any errors.

In [5]:
response = es.get(index="my_index", id="sKYOPJoBbrc3W2pBVbKQ")

In [6]:
pprint(response.body)

{'_id': 'sKYOPJoBbrc3W2pBVbKQ',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 0,
 '_source': {'created_on': '2024-09-22',
             'text': 'This is the first sample document text.',
             'title': 'Sample Title 1'},
 '_version': 1,
 'found': True}


This example shows that the get operation fails when providing it with an ID that does not exist in the index.

In [7]:
try:
    response = es.get(index='my_index', id="id")
except Exception as e:
    print(e)

NotFoundError(404, "{'_index': 'my_index', '_id': 'id', 'found': False}")
